# Import libraries

Not sure we will need all these libraries for kedro.

In [1]:
# Importing modules for data science and visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 180 # Quality of all figures in notebook

# imports for Natural Language  Processing
import re
import os
import nltk
import string
import time
import unicodedata
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from html.parser import HTMLParser
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import remove_stopwords
import emoji
from emoji import emojize
from collections import Counter
import itertools
import pickle

# Classification Models
import xgboost
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

# Performance Evaluation
from sklearn.pipeline import Pipeline
#from pipelinehelper import PipelineHelper
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix

# Import library for train test split
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

# Wordcloud
from PIL import Image
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt


#Resampling techniques
from collections import Counter 
from sklearn.utils import resample



import warnings
warnings.filterwarnings("ignore")

In [2]:
import texthero as hero

In [3]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\Adetunji
[nltk_data]     Emmanuel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
def import_dataset(path: str) -> pd.DataFrame:
    """
    Data read_csvtion, importing `csv` file only
    Args:
        path: string like `file` address in the directory
    Return:
        df: (Dataframe) output imported csv file as pandas Dataframe
    """
    train_df = pd.read_csv(path, sep='\t')
    return train_df

In [3]:
data = import_dataset('osdg-community-dataset-v21-09-30.csv')
data.head(5)

,doi,text_id,text,sdg,labels_negative,labels_positive,agreement
0,10.6027/9789289342698-7-en,00021941702cd84171ff33962197ca1f,"From a gender perspective, Paulgaard points ou...",5,1,7,0.750000
1,10.18356/eca72908-en,00028349a7f9b2485ff344ae44ccfd6b,Labour legislation regulates maximum working h...,11,2,1,0.333333
2,10.1787/9789264289062-4-en,0004eb64f96e1620cd852603d9cbe4d4,The average figure also masks large difference...,3,1,6,0.714286
3,10.1787/5k9b7bn5qzvd-en,0006a887475ccfa5a7f5f51d4ac83d02,The extent to which they are akin to corruptio...,3,1,2,0.333333
4,10.1787/9789264258211-6-en,0006d6e7593776abbdf4a6f985ea6d95,A region reporting a higher rate will not earn...,3,2,2,0.000000


# 1. Data preprocessing 

In [4]:
def clean_agreement(data: pd.DataFrame) -> pd.DataFrame:
    '''
    This function takes in a dataframe and filters out the rows with negative label higher than positive label,
    and have an agreement score less than 0.4
    
    Args: 
        Source training data
        
    Returns:
        Filtered data that has high positive community agreement with SDG labels
        
        
    '''
    data = data.loc[(data['labels_negative'] < data['labels_positive']) & (data['agreement'] >= 0.4)]
    return data

In [5]:
def missing_data(df: pd.DataFrame) ->pd.DataFrame:
    """
    
    """
    df1 = df.dropna(subset=['text'])
    df2 = df1.dropna(subset=['sdg'])
    return df2

#### Secondary functions used to pre-process the text data

Can we create more functions that clean data in a more specialized way?

In [6]:
def clean_article(text: str) -> str:
    
    """Converts apostrophe suffixes to words, replace webpage links with url, annotate 
    hashtags and mentions, remove a selection of punctuation, and convert all words to lower case.
    Args:
        text: 'text' article or sentence to be cleaned
    Returns:
        clean_text: clean text rid of noise 
    """
    def remove_extras(post):
        punc_numbers = string.punctuation + '0123456789'
        return ''.join([l for l in post if l not in punc_numbers])
    #Lower case
    text = text.lower()
    #Removal of Punctuation
    text = remove_extras(text)
    return text

In [10]:
def _lemmatize(text: str) -> str:
    """Lemmatize text.
    Args:
        text (String): sentence containing 'text' to lemmatize (stemming)
    Returns:
        text (String): sentence with converted 'text' into lemmatized word(s)  
    """
    # removing any form of hyper link(s)
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words and len(word)>2]
    return ' '.join(text)

#### Primary function to pre-process data

In [8]:
def preprocess_data(data: pd.DataFrame) -> pd.DataFrame:
    '''
    Preprocess data.
    
     Args:
        data: Full (all columns) training cleaned data according to agreement score
        
     Returns:
        Processed text data as pandas dataframe.
        
    '''
    # removing missing rows in the important columns
    data = missing_data(data)
    #apply text hero
    data['text'] = data['text'].apply(clean_article)
    #lemmatize the text
    data['text'] = data['text'].apply(_lemmatize)


    return data

In [9]:
data = clean_agreement(data)
data.head(3)

,doi,text_id,text,sdg,labels_negative,labels_positive,agreement
0,10.6027/9789289342698-7-en,00021941702cd84171ff33962197ca1f,"From a gender perspective, Paulgaard points ou...",5,1,7,0.750000
2,10.1787/9789264289062-4-en,0004eb64f96e1620cd852603d9cbe4d4,The average figure also masks large difference...,3,1,6,0.714286
5,10.1787/5js4xfgl4ks0-en,000b54717f2deea5d99055b4c1c2bf5a,These findings are consistent with previous wo...,10,2,5,0.428571


In [12]:
data = preprocess_data(data)
data.head(3)

,doi,text_id,text,sdg,labels_negative,labels_positive,agreement
0,10.6027/9789289342698-7-en,00021941702cd84171ff33962197ca1f,gender perspective paulgaard point labour mark...,5,1,7,0.750000
2,10.1787/9789264289062-4-en,0004eb64f96e1620cd852603d9cbe4d4,average figure also mask large difference acro...,3,1,6,0.714286
5,10.1787/5js4xfgl4ks0-en,000b54717f2deea5d99055b4c1c2bf5a,finding consistent previous work examined diff...,10,2,5,0.428571


#### Data Balancing

In [13]:
# Creating a function to upscale and balance the dataset
def data_balancing(df_input: pd.DataFrame) -> pd.DataFrame:
    """
        Dataset balancing for all target variable to be equal in frequency.
    Args:
        `df` (DataFrame): pd.Series containing the target variable
    Return:
        df (DataFrame): dataframe with resample and balance dataset by upscaling.
    """
    df = df_input.copy()
    # setting the maximum size for each category in 'sdg'
    class_size = int(df.sdg.value_counts().max()) 

    target_size = df.sdg.value_counts() # getting category name and their size
    appended_target = [] # creating an empty list to append all category after resampling

    # Creating a for-loop to resample and append to a list
    for index, size in target_size.items():
        if size < class_size: # setting condition to check if to downsample or upsample
            temp_pd = resample(df[df['sdg']==index],
                              replace=True, # sample with replacement
                              n_samples=class_size, # match number in majority class
                              random_state=27)
        else:
            temp_pd = resample(df[df['sdg']==index],
                              replace=False, # sample with replacement (no need to duplicate observations)
                              n_samples=class_size, # match number in minority class
                              random_state=27)
    # Appending each category after resampling
        appended_target.append(temp_pd)
        
    # Creating a new dataframe and viewing
    df_resampled = pd.concat(appended_target, axis=0)
    
    return df_resampled

In [14]:
# Resampling the dataset to balance each target
data = data_balancing(data)

The problem with jupyter notebooks -> How do we know when we have run each cell and in what order did we run each cell?

# 2. Data Science

In [15]:
# utilities
from typing import List, Dict
import pandas as pd
import numpy as np
import logging

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score

#### Split data

Parameters

In [16]:
#this dict will be stored in the paramter.yml file in Kedro

parameters = {'test_size': 0.2,
              'random_state': 42,
             'ngram_range_min': 1,
              'ngram_range_max': 2,
              'max_features': 100000,
              'max_iter': 1000,
              'C': 1
             }

In [17]:
def split_data(data: pd.DataFrame, parameters: Dict) -> List:
    '''
    Splits data into training and test set.
    
     Args:
        data: Source processed data.
        parameters: Parameters defined in parameter.yml.
    
     Returns:
        A list containing split data.
        
    '''
    #data = _handle_empty(data) ... what happens if there is missing data? We could create a function for this as well
    X = data['text'].values
    y = data['sdg'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=parameters['test_size'], 
                                                        random_state=parameters['random_state'])
    return [X_train, X_test, y_train, y_test]

In [18]:
# spliting the data set to train and test set
X_train, X_test, y_train, y_test = split_data(data, parameters )

#### Vectorizer

In [19]:
def vectorize_text(X_train: np.ndarray, X_test: np.ndarray, parameters: Dict) -> List:
    '''
    Vectorize text column in train and test set.
    
     Args:
        X_train: Training text data.
        X_test: Testing text data.
        parameters: Parameters defined in parameter.yml.   
       
     Returns:
        A list containing vectorized train and test feature sets. 
        
    '''


    #E.g Tfid
    vectorizer = TfidfVectorizer(ngram_range=(parameters['ngram_range_min'],
                                              parameters['ngram_range_max']),
                                 max_features=parameters['max_features'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)


    return [X_train, X_test]

In [20]:
X_train, X_test = vectorize_text(X_train, X_test, parameters)

#### Training the model

In [21]:
def train_model(X_train: np.ndarray, y_train: np.ndarray, parameters: Dict) -> LinearSVC:
    '''
    Train the Linear SVC model.
    
     Args:
        X_train: Vectorized training text data.
        y_train: Training data for SDG labels.
        parameters: Parameters defined in parameter.yml.
        
     Returns:
        Trained model.
        
    '''
    classifier = LinearSVC(max_iter=parameters['max_iter'], C=parameters['C'],
                           random_state=parameters['random_state'])
    classifier.fit(X_train, y_train)
    return classifier

In [22]:
classifier = train_model(X_train,y_train, parameters)

#### Evaluate model

In [23]:
def evaluate_model(X_test: np.ndarray, y_test: np.ndarray, classifier: LinearSVC):
    '''
    Generate and log classification report for test data.
    
     Args:
        X_test: Vectorized test text data.
        y_test: Test data for SDG.
        classifier: Trained model.
        
    '''
    y_pred = classifier.predict(X_test)
    
    score = f1_score(y_test, y_pred, average='weighted')
    logger = logging.getLogger(__name__)
    logger.info("Model has an f1 score (weighted) of %.3f on test data.", score)
    #logger.info(classification_report(y_test, y_pred)) 
    
    #in kedro we will not return score, only log what we have done
    return score

In [24]:
score  = evaluate_model(X_test,y_test, classifier )
print(score)

0.973020202634326


What can you help with?
1. Include data balancing
2. More and better specialized functions for text pre-processing. We cant see what is happening under text hero
3. Create a function that handles missing data
